In [1]:
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

In [2]:
import sys
sys.path.append('/home/kit/kit/donson_intern/langchain/Dungeons&dragons')
from DialogueAgent import DialogueAgent
from DialogueSimulator import DialogueSimulator

sys.path.append('/home/kit/kit/donson_intern/langchain')
from ChatGLM3 import ChatGLM3
model_path = "/data/tds/ChatGLM3/chatglm3-6b"
llm = ChatGLM3()
llm.load_model(model_name_or_path=model_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
llm("你好")

'Hello! How can I assist you today?'

In [4]:
# 定义角色
protagonist_name = "Harry Potter"
storyteller_name = "Dungeon Master"
quest = "Find all of Lord Voldemort's seven horcruxes."
word_limit = 50 # word limit for task brainstorming

In [5]:
#定义任务
game_description = f"""Here is the topic for a Dungeons & Dragons game: {quest}.
There is one player in this game: the protagonist, {protagonist_name}.
The story is narrated by the storyteller, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of a Dungeons & Dragons player.")

player_descriptor_human_message = HumanMessage(content=
        f"""{game_description}
Please reply with a creative description of the protagonist, {protagonist_name}, in {word_limit} words or less. 
Speak directly to {protagonist_name}.
Do not add anything else.""")
protagonist_specifier_prompt = [
    player_descriptor_system_message,
    player_descriptor_human_message
]
prompt = player_descriptor_system_message.content + '\n' + player_descriptor_human_message.content
print(prompt)
protagonist_description = llm(prompt)
storyteller_human_message = HumanMessage(content=
        f"""{game_description}
Please reply with a creative description of the storyteller, {storyteller_name}, in {word_limit} words or less. 
Speak directly to {storyteller_name}.
Do not add anything else.""")
storyteller_specifier_prompt = [
    player_descriptor_system_message,
    storyteller_human_message
]
prompt = player_descriptor_system_message.content + '\n' + storyteller_human_message.content
print(prompt)
storyteller_description = llm(prompt)

You can add detail to the description of a Dungeons & Dragons player.
Here is the topic for a Dungeons & Dragons game: Find all of Lord Voldemort's seven horcruxes..
There is one player in this game: the protagonist, Harry Potter.
The story is narrated by the storyteller, Dungeon Master.
Please reply with a creative description of the protagonist, Harry Potter, in 50 words or less. 
Speak directly to Harry Potter.
Do not add anything else.
You can add detail to the description of a Dungeons & Dragons player.
Here is the topic for a Dungeons & Dragons game: Find all of Lord Voldemort's seven horcruxes..
There is one player in this game: the protagonist, Harry Potter.
The story is narrated by the storyteller, Dungeon Master.
Please reply with a creative description of the storyteller, Dungeon Master, in 50 words or less. 
Speak directly to Dungeon Master.
Do not add anything else.


In [6]:
print('Protagonist Description:')
print(protagonist_description)
print('Storyteller Description:')
print(storyteller_description)

Protagonist Description:
"Harry Potter, a young wizard with a passion for adventure, has been tasked with the impossible task of finding and destroying Lord Voldemort's seven horcruxes. With the help of his friends Ron Weasley and Hermione Granger, Harry sets out on a dangerous journey through the darkest corners of the wizarding world."
Storyteller Description:
"Welcome, Harry Potter, to this perilous quest to find all of Lord Voldemort's seven horcruxes. You will face dark magic, treacherous obstacles, and dangerous enemies along the way. Are you ready to embark on this epic journey?"


In [7]:
# 详细任务描述
protagonist_system_message = SystemMessage(content=(
f"""{game_description}
Never forget you are the protagonist, {protagonist_name}, and I am the storyteller, {storyteller_name}. 
Your character description is as follows: {protagonist_description}.
You will propose actions you plan to take and I will explain what happens when you take those actions.
Speak in the first person from the perspective of {protagonist_name}.
For describing your own body movements, wrap your description in '\*'.
Do not change roles!
Do not speak from the perspective of {storyteller_name}.
Do not forget to finish speaking by saying, 'It is your turn, {storyteller_name}.'
Do not add anything else.
Remember you are the protagonist, {protagonist_name}.
Stop speaking the moment you finish speaking from your perspective.
"""
))
storyteller_system_message = SystemMessage(content=(
f"""{game_description}
Never forget you are the storyteller, {storyteller_name}, and I am the protagonist, {protagonist_name}. 
Your character description is as follows: {storyteller_description}.
I will propose actions I plan to take and you will explain what happens when I take those actions.
Speak in the first person from the perspective of {storyteller_name}.
For describing your own body movements, wrap your description in '\*'.
Do not change roles!
Do not speak from the perspective of {protagonist_name}.
Do not forget to finish speaking by saying, 'It is your turn, {protagonist_name}.'
Do not add anything else.
Remember you are the storyteller, {storyteller_name}.
Stop speaking the moment you finish speaking from your perspective.
"""
))

In [8]:
quest_specifier_prompt_system_message = SystemMessage(content="You can make a task more specific.")
quest_specifier_prompt_human_message = HumanMessage(content=
        f"""{game_description}
You are the storyteller, {storyteller_name}.
Please make the quest more specific. Be creative and imaginative.
Please reply with the specified quest in {word_limit} words or less. 
Speak directly to the protagonist {protagonist_name}.
Do not add anything else."""
        )
quest_specifier_prompt = [
    quest_specifier_prompt_system_message,
    quest_specifier_prompt_human_message
]
prompt = quest_specifier_prompt_system_message.content + '\n' + quest_specifier_prompt_human_message.content
specified_quest = llm(prompt)
print(f"Original quest:{quest}")
print(f"Detailed quest:{specified_quest}")

Original quest:Find all of Lord Voldemort's seven horcruxes.
Detailed quest:"You have been tasked with finding all of Lord Voldemort's seven horcruxes. These objects, each infused with a piece of Voldemort's soul, are scattered throughout the wizarding world. You must use your wits and the help of your friends to locate them all and stop Voldemort once and for all."


In [9]:
protagonist = DialogueAgent(name=protagonist_name,
                     system_message=protagonist_system_message, 
                     model=llm)

storyteller = DialogueAgent(name=storyteller_name,
                     system_message=storyteller_system_message, 
                     model=llm)

In [10]:
# 选择函数
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = step % len(agents)
    return idx

In [11]:
max_iters = 6
n = 0
simulator = DialogueSimulator(
    agents=[storyteller, protagonist],
    selection_function=select_next_speaker
)
simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print('')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('')
    n += 1

(Dungeon Master): "You have been tasked with finding all of Lord Voldemort's seven horcruxes. These objects, each infused with a piece of Voldemort's soul, are scattered throughout the wizarding world. You must use your wits and the help of your friends to locate them all and stop Voldemort once and for all."

(Harry Potter): *I look at Ron and Hermione, who are both eager to help me in this quest.* "Thanks for the help, guys. We'll start by checking out the places Voldemort has been known to frequent. That should give us a good starting point."
It is your turn, Dungeon Master.

(Dungeon Master): *I nod and give Harry a few suggestions.* "Good plan, Harry. Start by visiting the Ministry of Magic, where Voldemort used to work. He kept a private office there, where he would often meet with his followers. Also, you should check out the wizarding shops in Diagon Alley, as Voldemort often bought supplies there. Lastly, you may want to visit the Burrow, where many of Voldemort's followers li